In [2]:
import numpy as np
import pandas as pd
from pvlib.location import Location

from pyrealm.pmodel import PModel, PModelEnvironment, SubdailyPModel

In [3]:
forcing_data = pd.read_csv("merged_BE-Vie_data.csv")
forcing_data["time"] = pd.to_datetime(forcing_data["time"])
forcing_data

,time,ppfd,vpd,tc,co2,patm,gpp_fluxnet,LAI,fapar,GPP_JAMES
0,2014-01-01 00:00:00,2.0,52.2,3.33,418.407,95300.0,0.0,1.942698,0.523071,NaN
1,2014-01-01 00:30:00,2.0,0.0,2.71,418.836,95300.0,0.0,1.942698,0.523071,NaN
2,2014-01-01 01:00:00,2.0,0.0,2.91,418.729,95300.0,0.0,1.942698,0.523071,NaN
3,2014-01-01 01:30:00,2.0,0.0,3.12,418.472,95300.0,0.0,1.942698,0.523071,NaN
4,2014-01-01 02:00:00,2.0,0.0,3.21,418.395,95300.0,0.0,1.942698,0.523071,NaN
...,...,...,...,...,...,...,...,...,...,...
17515,2014-12-31 21:30:00,0.0,0.0,-1.24,394.193,97200.0,0.0,1.953821,0.444753,NaN
17516,2014-12-31 22:00:00,0.0,0.0,-1.46,396.109,97200.0,0.0,1.953821,0.444753,NaN
17517,2014-12-31 22:30:00,0.0,0.0,-1.70,395.776,97200.0,0.0,1.953821,0.444753,NaN
17518,2014-12-31 23:00:00,0.0,0.0,-1.93,394.601,97200.0,0.0,1.953821,0.444753,NaN


In [4]:
pv_location = Location(
    latitude=50.30493, longitude=5.99812, tz="Etc/GMT+1", altitude=493, name="BE-Vie"
)
solar_position = pv_location.get_solarposition(forcing_data["time"])
solar_position

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
time,,,,,,
2014-01-01 00:00:00,152.420122,152.420122,-62.420122,-62.420122,10.321119,-3.304259
2014-01-01 00:30:00,150.978178,150.978178,-60.978178,-60.978178,24.585260,-3.314157
2014-01-01 01:00:00,148.506258,148.506258,-58.506258,-58.506258,37.398486,-3.324052
2014-01-01 01:30:00,145.237642,145.237642,-55.237642,-55.237642,48.541226,-3.333946
2014-01-01 02:00:00,141.393309,141.393309,-51.393309,-51.393309,58.150628,-3.343837
...,...,...,...,...,...,...
2014-12-31 21:30:00,142.950364,142.950364,-52.950364,-52.950364,305.337753,-3.135913
2014-12-31 22:00:00,146.596355,146.596355,-56.596355,-56.596355,315.510680,-3.145789
2014-12-31 22:30:00,149.585960,149.585960,-59.585960,-59.585960,327.298609,-3.155662


In [5]:
# Add solar elevation as radians to the forcing dataframe
forcing_data["solar_zenith"] = np.deg2rad(solar_position["zenith"].to_numpy())
forcing_data["solar_elevation"] = np.deg2rad(solar_position["elevation"].to_numpy())
data = forcing_data.loc[(forcing_data["time"] == "2014-08-01 12:30:00")]

In [20]:
class TwoLeafIrradiance:
    def __init__(
        self,
        beta_angle,
        PPFD,
        LAI,
        PATM,
        solar_obscurity_angle=np.deg2rad(1),
        PA0=101325,
        fa=0.426,
        sigma=0.15,
        rho_cd=0.036,
        kd_prime=0.719,
    ):
        """Partition solar irradiance

        The references in parentheses e.g. (A23) refer to the equation sequence in
        deP&F 1997 and recreated here.
        """

        # Store key inputs
        self.solar_obscurity_angle = solar_obscurity_angle
        self.beta_angle = beta_angle

        # beam extinction coefficient, clipped below solar_obscurity_angle
        self.kb = np.where(
            beta_angle > solar_obscurity_angle, 0.5 / np.sin(self.beta_angle), 30
        )

        # beam and scattered-beam extinction coefficient
        kb_prime = np.where(
            beta_angle > solar_obscurity_angle, 0.46 / np.sin(self.beta_angle), 27
        )

        m = (PATM / PA0) / np.sin(beta_angle)

        fd = (1 - 0.72**m) / (1 + ((0.72**m) * ((1 / fa) - 1)))

        # beam irradiance horizontal leaves (A20)
        rho_h = (1 - np.sqrt(1 - sigma)) / (1 + np.sqrt(1 - sigma))

        # beam irradiance uniform leaf-angle distribution (A19)
        rho_cb = 1 - np.exp(-2 * rho_h * self.kb / (1 + self.kb))

        # diffuse radiation, truncating at zero
        I_d = np.clip(PPFD * fd, a_min=0, a_max=np.inf)

        # beam irradiance
        I_b = PPFD * (1 - fd)

        # scattered beam irradiance (A8)
        I_bs = I_b * (1 - rho_cb) * kb_prime * np.exp(-kb_prime * LAI) - (
            1 - sigma
        ) * self.kb * np.exp(-self.kb * LAI)

        # Irradiance absorbed by the whole canopy (Eqn 13)
        self.I_c = (1 - rho_cb) * I_b * (1 - np.exp(-kb_prime * LAI)) + (
            1 - rho_cd
        ) * I_d * (1 - np.exp(-kb_prime * LAI))

        # for the sunlit leaves, the components are (Eqn 20):

        self.Isun_beam = I_b * (1 - sigma) * (1 - np.exp(-self.kb * LAI))
        self.Isun_diffuse = (
            I_d
            * (1 - rho_cd)
            * (1 - np.exp(-(kd_prime + self.kb) * LAI))
            * kd_prime
            / (kd_prime + self.kb)
        )
        self.Isun_scattered = I_b * (
            (
                (1 - rho_cb)
                * (1 - np.exp(-(kb_prime + self.kb) * LAI))
                * kb_prime
                / (kb_prime + self.kb)
            )
            - ((1 - sigma) * (1 - np.exp(-2 * self.kb * LAI)) / 2)
        )

        # Irradiance absorbed by the sunlit fraction of the canopy (Eqn 20):
        self.I_csun = self.Isun_beam + self.Isun_diffuse + self.Isun_scattered

        # Irradiance absorbed by the shaded fraction of the canopy (Eqn 21):
        # and including a clause to exclude the hours of obscurity
        self.I_cshade = np.where(
            beta_angle > solar_obscurity_angle, self.I_c - self.I_csun, 0
        )

        self.rho_h = rho_h
        self.rho_cb = rho_cb
        self.I_d = I_d
        self.I_b = I_b
        self.I_bs = I_bs
        self.kb_prime = kb_prime

In [21]:
two_leaf_irrad = TwoLeafIrradiance(
    beta_angle=data["solar_elevation"].to_numpy(),
    PPFD=data["ppfd"].to_numpy(),
    LAI=data["LAI"].to_numpy(),
    PATM=data["patm"].to_numpy(),
)

In [23]:
print(f"kb = {two_leaf_irrad.kb}")
print(f"kb_prime = {two_leaf_irrad.kb_prime}")
print(f"rho_h = {two_leaf_irrad.rho_h}")
print(f"rho_cb = {two_leaf_irrad.rho_cb}")
print(f"I_d = {two_leaf_irrad.I_d}")
print(f"I_b = {two_leaf_irrad.I_b}")
print(f"I_bs = {two_leaf_irrad.I_bs}")
print(f"I_c = {two_leaf_irrad.I_c}")
print(f"Isun_beam = {two_leaf_irrad.Isun_beam}")
print(f"Isun_diffuse = {two_leaf_irrad.Isun_diffuse}")
print(f"Isun_scattered = {two_leaf_irrad.Isun_scattered}")
print(f"beta_angle = {two_leaf_irrad.beta_angle}")

kb = [0.60119491]
kb_prime = [0.55309931]
rho_h = 0.04060739027615024
rho_cb = [0.03003319]
I_d = [136.29450038]
I_b = [714.70549962]
I_bs = [87.564122]
I_c = [636.08707023]
Isun_beam = [485.32862357]
Isun_diffuse = [69.44255865]
Isun_scattered = [25.43894747]
beta_angle = [0.98212117]


In [24]:
pmod_env = PModelEnvironment(
    tc=data["tc"].to_numpy(),
    patm=data["patm"].to_numpy(),
    co2=data["co2"].to_numpy(),
    vpd=data["vpd"].to_numpy(),
)

In [31]:
standard_pmod = PModel(pmod_env, kphio=1 / 8)
standard_pmod.estimate_productivity(
    fapar=data["fapar"].to_numpy(), ppfd=data["ppfd"].to_numpy()
)

In [32]:
class TwoLeafAssimilation:
    def __init__(
        self, pmodel: PModel | SubdailyPModel, irrad: TwoLeafIrradiance, LAI: np.array
    ):
        # A load of needless inconsistencies in the object attribute names - to be resolved!
        vcmax_pmod = (
            pmodel.vcmax if isinstance(pmodel, PModel) else pmodel.subdaily_vcmax
        )
        vcmax25_pmod = (
            pmodel.vcmax25 if isinstance(pmodel, PModel) else pmodel.subdaily_vcmax25
        )
        optchi_obj = pmodel.optchi if isinstance(pmodel, PModel) else pmodel.optimal_chi
        core_const = (
            pmodel.core_const if isinstance(pmodel, PModel) else pmodel.env.core_const
        )

        # Generate extinction coefficients to express the vertical gradient in photosynthetic
        # capacity after the equation provided in Figure 10 of Lloyd et al. (2010):

        kv_Lloyd = np.exp(
            0.00963 * vcmax_pmod - 2.43
        )  # KB: Here I opt for vcmax rather than Vcmax25

        # Calculate carboxylation in the two partitions at standard conditions
        self.Vmax25_canopy = LAI * vcmax25_pmod * ((1 - np.exp(-kv_Lloyd)) / kv_Lloyd)
        self.Vmax25_sun = (
            LAI
            * vcmax25_pmod
            * ((1 - np.exp(-kv_Lloyd - irrad.kb * LAI)) / (kv_Lloyd + irrad.kb * LAI))
        )
        self.Vmax25_shade = self.Vmax25_canopy - self.Vmax25_sun

        # Convert carboxylation rates to ambient temperature using an Arrhenius function
        self.Vmax_sun = self.Vmax25_sun * np.exp(
            64800 * (pmodel.env.tc - 25) / (298 * 8.314 * (pmodel.env.tc + 273))
        )
        self.Vmax_shade = self.Vmax25_shade * np.exp(
            64800 * (pmodel.env.tc - 25) / (298 * 8.314 * (pmodel.env.tc + 273))
        )

        # Now the photosynthetic estimates as V_cmax * mc
        self.Av_sun = self.Vmax_sun * optchi_obj.mc
        self.Av_shade = self.Vmax_shade * optchi_obj.mc

        ## Jmax estimates for sun and shade;
        self.Jmax25_sun = 29.1 + 1.64 * self.Vmax25_sun  # Eqn 31, after Wullschleger
        self.Jmax25_shade = 29.1 + 1.64 * self.Vmax25_shade

        # Temperature correction (Mengoli 2021 Eqn 3b); relevant temperatures given in Kelvin
        self.Jmax_sun = self.Jmax25_sun * np.exp(
            (43990 / 8.314) * (1 / 298 - 1 / (pmodel.env.tc + 273))
        )
        self.Jmax_shade = self.Jmax25_shade * np.exp(
            (43990 / 8.314) * (1 / 298 - 1 / (pmodel.env.tc + 273))
        )

        # Calculate J and Aj for each partition
        self.J_sun = (
            self.Jmax_sun
            * irrad.I_csun
            * (1 - 0.15)
            / (irrad.I_csun + 2.2 * self.Jmax_sun)
        )
        self.J_shade = (
            self.Jmax_shade
            * irrad.I_cshade
            * (1 - 0.15)
            / (irrad.I_cshade + 2.2 * self.Jmax_shade)
        )

        self.Aj_sun = (self.J_sun / 4) * optchi_obj.mj
        self.Aj_shade = (self.J_shade / 4) * optchi_obj.mj

        # Calculate the assimilation in each partition as the minimum of Aj and Av,
        # in both cases clipping when the sun is below the angle of obscurity
        Acanopy_sun = np.minimum(self.Aj_sun, self.Av_sun)
        self.Acanopy_sun = np.where(
            irrad.beta_angle < irrad.solar_obscurity_angle, 0, Acanopy_sun
        )
        Acanopy_shade = np.minimum(self.Aj_shade, self.Av_shade)
        self.Acanopy_shade = np.where(
            irrad.beta_angle < irrad.solar_obscurity_angle, 0, Acanopy_shade
        )

        self.gpp = core_const.k_c_molmass * self.Acanopy_sun + self.Acanopy_shade

In [33]:
two_leaf_assim_standard = TwoLeafAssimilation(
    irrad=two_leaf_irrad, pmodel=standard_pmod, LAI=data["LAI"].to_numpy()
)

In [39]:
print(f"Vmax25_canopy = {two_leaf_assim_standard.Vmax25_canopy}")
print(f"Vmax25_sun = {two_leaf_assim_standard.Vmax25_sun}")
print(f"Vmax25_shade = {two_leaf_assim_standard.Vmax25_shade}")
print(f"Vmax_sun = {two_leaf_assim_standard.Vmax_sun}")
print(f"Vmax_shade = {two_leaf_assim_standard.Vmax_shade}")
print(f"Av_sun = {two_leaf_assim_standard.Av_sun}")
print(f"Av_shade = {two_leaf_assim_standard.Av_shade}")
print(f"Jmax25_sun = {two_leaf_assim_standard.Jmax25_sun}")
print(f"Jmax25_shade = {two_leaf_assim_standard.Jmax25_shade}")
print(f"J_sun = {two_leaf_assim_standard.J_sun}")
print(f"J_shade = {two_leaf_assim_standard.J_shade}")
print(f"Jmax_sun = {two_leaf_assim_standard.Jmax_sun}")
print(f"Jmax_shade = {two_leaf_assim_standard.Jmax_shade}")
print(f"Aj_sun = {two_leaf_assim_standard.Aj_sun}")
print(f"Aj_shade = {two_leaf_assim_standard.Aj_shade}")
print(f"Acanopy_sun = {two_leaf_assim_standard.Acanopy_sun}")
print(f"Acanopy_shade = {two_leaf_assim_standard.Acanopy_shade}")
print(f"gpp = {two_leaf_assim_standard.gpp}")

Vmax25_canopy = [220.32138308]
Vmax25_sun = [112.19495779]
Vmax25_shade = [108.12642529]
Vmax_sun = [88.33176716]
Vmax_shade = [85.12858699]
Av_sun = [23.7468972]
Av_shade = [22.88576204]
Jmax25_sun = [213.09973077]
Jmax25_shade = [206.42733748]
J_sun = [91.28498756]
J_shade = [18.85937627]
Jmax_sun = [181.16701827]
Jmax_shade = [175.49447428]
Aj_sun = [15.25224592]
Aj_shade = [3.15109694]
Acanopy_sun = [15.25224592]
Acanopy_shade = [3.15109694]
gpp = [186.34124706]
